## 1. Import Libraries and Setup

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Clustering algorithms
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist

# Metrics
from sklearn.metrics import silhouette_score, silhouette_samples, calinski_harabasz_score, davies_bouldin_score

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load dataset
df = pd.read_csv('../../../data/data/CC GENERAL.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data information
print("Dataset Information:")
print(df.info())
print("\n" + "="*50)
print("\nStatistical Summary:")
df.describe()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)

print("Missing Values Summary:")
print(missing_df[missing_df['Missing_Count'] > 0])

# Visualize missing values
if missing_df['Missing_Count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    sns.barplot(x=missing_df[missing_df['Missing_Count'] > 0].index, 
                y=missing_df[missing_df['Missing_Count'] > 0]['Percentage'])
    plt.title('Missing Values Percentage by Feature', fontsize=14, fontweight='bold')
    plt.xlabel('Features')
    plt.ylabel('Missing %')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 3. Data Preprocessing

In [ ]:
# Handle missing values
df_clean = df.copy()

# Fill missing values with median for numerical columns
for col in df_clean.columns:
    if df_clean[col].dtype in ['float64', 'int64']:
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

# Remove CUST_ID as it's not useful for clustering
if 'CUST_ID' in df_clean.columns:
    customer_ids = df_clean['CUST_ID'].copy()
    df_clean = df_clean.drop('CUST_ID', axis=1)
else:
    customer_ids = pd.Series(range(len(df_clean)), name='CUST_ID')

print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Remaining missing values: {df_clean.isnull().sum().sum()}")

In [ ]:
# Feature distributions
fig, axes = plt.subplots(6, 3, figsize=(18, 20))
axes = axes.ravel()

for idx, col in enumerate(df_clean.columns[:18]):
    axes[idx].hist(df_clean[col], bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_title(col, fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')

plt.tight_layout()
plt.suptitle('Feature Distributions', fontsize=16, fontweight='bold', y=1.001)
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(16, 14))
correlation = df_clean.corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Standardize features
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_clean),
    columns=df_clean.columns
)

print(f"Scaled data shape: {df_scaled.shape}")
print(f"\nScaled data statistics:")
print(df_scaled.describe())

## 4. Dimensionality Reduction with PCA

In [ ]:
# Apply PCA
pca = PCA(n_components=df_scaled.shape[1])
pca_components = pca.fit_transform(df_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print(f"Variance explained by first 5 components: {cumulative_variance[4]:.3f}")
print(f"Variance explained by first 3 components: {cumulative_variance[2]:.3f}")
print(f"Variance explained by first 2 components: {cumulative_variance[1]:.3f}")

In [ ]:
# Visualize explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Individual variance
ax1.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Principal Component', fontsize=12)
ax1.set_ylabel('Explained Variance Ratio', fontsize=12)
ax1.set_title('Variance Explained by Each Principal Component', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Cumulative variance
ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
         marker='o', linestyle='-', linewidth=2, markersize=6)
ax2.axhline(y=0.90, color='r', linestyle='--', label='90% Variance')
ax2.axhline(y=0.95, color='g', linestyle='--', label='95% Variance')
ax2.set_xlabel('Number of Components', fontsize=12)
ax2.set_ylabel('Cumulative Explained Variance', fontsize=12)
ax2.set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create PCA datasets for visualization
pca_2d = PCA(n_components=2)
pca_2d_data = pca_2d.fit_transform(df_scaled)

pca_3d = PCA(n_components=3)
pca_3d_data = pca_3d.fit_transform(df_scaled)

print(f"2D PCA data shape: {pca_2d_data.shape}")
print(f"3D PCA data shape: {pca_3d_data.shape}")

## 5. K-Means Clustering

### 5.1 Elbow Method

In [ ]:
# Elbow method
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(df_scaled, kmeans.labels_))

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Elbow curve
ax1.plot(K_range, inertias, marker='o', linestyle='-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
ax1.set_title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Silhouette scores
ax2.plot(K_range, silhouette_scores, marker='s', linestyle='-', linewidth=2, markersize=8, color='orange')
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score by Number of Clusters', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSilhouette Scores:")
for k, score in zip(K_range, silhouette_scores):
    print(f"k={k}: {score:.4f}")

### 5.2 Apply K-Means with Optimal k

In [ ]:
# Apply K-Means with k=4 (typical optimal for this dataset)
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(df_scaled)

# Calculate metrics
silhouette_avg = silhouette_score(df_scaled, kmeans_labels)
calinski = calinski_harabasz_score(df_scaled, kmeans_labels)
davies_bouldin = davies_bouldin_score(df_scaled, kmeans_labels)

print(f"K-Means Clustering Results (k={optimal_k}):")
print(f"Silhouette Score: {silhouette_avg:.4f}")
print(f"Calinski-Harabasz Score: {calinski:.2f}")
print(f"Davies-Bouldin Score: {davies_bouldin:.4f}")
print(f"\nCluster Distribution:")
print(pd.Series(kmeans_labels).value_counts().sort_index())

In [ ]:
# 2D visualization
plt.figure(figsize=(14, 10))

scatter = plt.scatter(pca_2d_data[:, 0], pca_2d_data[:, 1], 
                     c=kmeans_labels, cmap='viridis', 
                     alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

# Plot cluster centers
centers_2d = pca_2d.transform(kmeans.cluster_centers_)
plt.scatter(centers_2d[:, 0], centers_2d[:, 1], 
           c='red', marker='X', s=300, edgecolors='black', linewidth=2, 
           label='Centroids')

plt.xlabel(f'First Principal Component ({pca_2d.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
plt.ylabel(f'Second Principal Component ({pca_2d.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
plt.title(f'K-Means Clustering (k={optimal_k}) - 2D PCA Visualization', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 3D interactive visualization
fig = go.Figure(data=[go.Scatter3d(
    x=pca_3d_data[:, 0],
    y=pca_3d_data[:, 1],
    z=pca_3d_data[:, 2],
    mode='markers',
    marker=dict(
        size=4,
        color=kmeans_labels,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Cluster"),
        line=dict(color='black', width=0.5)
    ),
    text=[f'Cluster: {label}' for label in kmeans_labels],
    hovertemplate='<b>Cluster %{text}</b><br>PC1: %{x:.2f}<br>PC2: %{y:.2f}<br>PC3: %{z:.2f}<extra></extra>'
)])

# Add centroids
centers_3d = pca_3d.transform(kmeans.cluster_centers_)
fig.add_trace(go.Scatter3d(
    x=centers_3d[:, 0],
    y=centers_3d[:, 1],
    z=centers_3d[:, 2],
    mode='markers',
    marker=dict(size=15, color='red', symbol='x', line=dict(color='black', width=2)),
    name='Centroids'
))

fig.update_layout(
    title=f'K-Means Clustering (k={optimal_k}) - 3D PCA Visualization',
    scene=dict(
        xaxis_title=f'PC1 ({pca_3d.explained_variance_ratio_[0]:.2%})',
        yaxis_title=f'PC2 ({pca_3d.explained_variance_ratio_[1]:.2%})',
        zaxis_title=f'PC3 ({pca_3d.explained_variance_ratio_[2]:.2%})'
    ),
    width=900,
    height=700
)

fig.show()

### 5.3 Silhouette Analysis

In [ ]:
# Silhouette analysis
silhouette_vals = silhouette_samples(df_scaled, kmeans_labels)

fig, ax = plt.subplots(figsize=(14, 8))
y_lower = 10

for i in range(optimal_k):
    cluster_silhouette_vals = silhouette_vals[kmeans_labels == i]
    cluster_silhouette_vals.sort()
    
    size_cluster_i = cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = plt.cm.viridis(float(i) / optimal_k)
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_vals,
                     facecolor=color, edgecolor=color, alpha=0.7)
    
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, f'Cluster {i}')
    y_lower = y_upper + 10

ax.set_xlabel('Silhouette Coefficient', fontsize=12)
ax.set_ylabel('Cluster', fontsize=12)
ax.set_title(f'Silhouette Analysis for K-Means (k={optimal_k})', fontsize=14, fontweight='bold')
ax.axvline(x=silhouette_avg, color='red', linestyle='--', linewidth=2, label=f'Average: {silhouette_avg:.3f}')
ax.set_yticks([])
ax.legend()
plt.tight_layout()
plt.show()

## 6. Hierarchical Clustering

In [ ]:
# Use a sample for dendrogram (full dataset would be too large)
sample_size = 1000
np.random.seed(42)
sample_indices = np.random.choice(len(df_scaled), sample_size, replace=False)
df_sample = df_scaled.iloc[sample_indices]

# Calculate linkage
linkage_matrix = linkage(df_sample, method='ward')

# Plot dendrogram
plt.figure(figsize=(18, 10))
dendrogram(linkage_matrix, 
          truncate_mode='lastp',
          p=30,
          leaf_rotation=90,
          leaf_font_size=10,
          show_contracted=True)
plt.xlabel('Sample Index or (Cluster Size)', fontsize=12)
plt.ylabel('Distance', fontsize=12)
plt.title('Hierarchical Clustering Dendrogram (Ward Linkage, Sample of 1000)', 
         fontsize=14, fontweight='bold')
plt.axhline(y=50, color='r', linestyle='--', linewidth=2, label='Cut at distance=50')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Apply hierarchical clustering on full dataset
hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage='ward')
hierarchical_labels = hierarchical.fit_predict(df_scaled)

# Calculate metrics
silhouette_hier = silhouette_score(df_scaled, hierarchical_labels)
calinski_hier = calinski_harabasz_score(df_scaled, hierarchical_labels)
davies_bouldin_hier = davies_bouldin_score(df_scaled, hierarchical_labels)

print(f"Hierarchical Clustering Results (k={optimal_k}):")
print(f"Silhouette Score: {silhouette_hier:.4f}")
print(f"Calinski-Harabasz Score: {calinski_hier:.2f}")
print(f"Davies-Bouldin Score: {davies_bouldin_hier:.4f}")
print(f"\nCluster Distribution:")
print(pd.Series(hierarchical_labels).value_counts().sort_index())

In [ ]:
# Visualize hierarchical clustering
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# 2D visualization
scatter1 = ax1.scatter(pca_2d_data[:, 0], pca_2d_data[:, 1], 
                       c=hierarchical_labels, cmap='viridis', 
                       alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
ax1.set_xlabel(f'First Principal Component ({pca_2d.explained_variance_ratio_[0]:.2%})', fontsize=12)
ax1.set_ylabel(f'Second Principal Component ({pca_2d.explained_variance_ratio_[1]:.2%})', fontsize=12)
ax1.set_title(f'Hierarchical Clustering (k={optimal_k})', fontsize=14, fontweight='bold')
plt.colorbar(scatter1, ax=ax1, label='Cluster')
ax1.grid(alpha=0.3)

# Compare with K-Means
scatter2 = ax2.scatter(pca_2d_data[:, 0], pca_2d_data[:, 1], 
                       c=kmeans_labels, cmap='viridis', 
                       alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
ax2.set_xlabel(f'First Principal Component ({pca_2d.explained_variance_ratio_[0]:.2%})', fontsize=12)
ax2.set_ylabel(f'Second Principal Component ({pca_2d.explained_variance_ratio_[1]:.2%})', fontsize=12)
ax2.set_title(f'K-Means Clustering (k={optimal_k})', fontsize=14, fontweight='bold')
plt.colorbar(scatter2, ax=ax2, label='Cluster')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. DBSCAN Clustering

In [ ]:
# Find optimal epsilon using k-distance graph
k = 4  # MinPts
distances = []
sample_for_eps = df_scaled.sample(n=min(2000, len(df_scaled)), random_state=42)

for i in range(len(sample_for_eps)):
    dist = cdist([sample_for_eps.iloc[i]], sample_for_eps, metric='euclidean')[0]
    dist = np.sort(dist)[k]  # k-th nearest neighbor
    distances.append(dist)

distances = sorted(distances, reverse=True)

plt.figure(figsize=(12, 6))
plt.plot(distances, linewidth=2)
plt.xlabel('Data Points (sorted by distance)', fontsize=12)
plt.ylabel(f'{k}-th Nearest Neighbor Distance', fontsize=12)
plt.title('K-Distance Graph for Epsilon Selection', fontsize=14, fontweight='bold')
plt.axhline(y=3.5, color='r', linestyle='--', linewidth=2, label='Suggested ε = 3.5')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Apply DBSCAN
dbscan = DBSCAN(eps=3.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(df_scaled)

# Calculate metrics (excluding noise points for silhouette)
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"DBSCAN Clustering Results:")
print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise} ({n_noise/len(dbscan_labels)*100:.2f}%)")

if n_clusters > 1:
    # Exclude noise points for silhouette calculation
    mask = dbscan_labels != -1
    if mask.sum() > 0:
        silhouette_dbscan = silhouette_score(df_scaled[mask], dbscan_labels[mask])
        print(f"Silhouette Score (excluding noise): {silhouette_dbscan:.4f}")

print(f"\nCluster Distribution:")
print(pd.Series(dbscan_labels).value_counts().sort_index())

In [ ]:
# Visualize DBSCAN results
plt.figure(figsize=(14, 10))

# Plot clusters
unique_labels = set(dbscan_labels)
colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Noise points in black
        col = 'gray'
        label = 'Noise'
        alpha = 0.3
        size = 20
    else:
        label = f'Cluster {k}'
        alpha = 0.6
        size = 50
    
    class_member_mask = (dbscan_labels == k)
    xy = pca_2d_data[class_member_mask]
    plt.scatter(xy[:, 0], xy[:, 1], c=[col], label=label, 
               alpha=alpha, s=size, edgecolors='black', linewidth=0.5)

plt.xlabel(f'First Principal Component ({pca_2d.explained_variance_ratio_[0]:.2%})', fontsize=12)
plt.ylabel(f'Second Principal Component ({pca_2d.explained_variance_ratio_[1]:.2%})', fontsize=12)
plt.title(f'DBSCAN Clustering (ε={dbscan.eps}, MinPts={dbscan.min_samples})', 
         fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Cluster Profiling and Analysis

In [ ]:
# Add cluster labels to original dataframe
df_clustered = df_clean.copy()
df_clustered['KMeans_Cluster'] = kmeans_labels
df_clustered['Hierarchical_Cluster'] = hierarchical_labels
df_clustered['DBSCAN_Cluster'] = dbscan_labels

print(f"Clustered dataset shape: {df_clustered.shape}")
df_clustered.head()

In [ ]:
# K-Means cluster profiles
print("K-MEANS CLUSTER PROFILES")
print("="*80)

cluster_profiles = df_clustered.groupby('KMeans_Cluster')[df_clean.columns].mean()
print("\nMean values by cluster:")
print(cluster_profiles.round(2))

print("\n" + "="*80)
print("\nCluster sizes:")
print(df_clustered['KMeans_Cluster'].value_counts().sort_index())

In [ ]:
# Visualize key features by cluster
key_features = ['BALANCE', 'PURCHASES', 'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS']

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.ravel()

for idx, feature in enumerate(key_features):
    if feature in df_clustered.columns:
        df_clustered.boxplot(column=feature, by='KMeans_Cluster', ax=axes[idx])
        axes[idx].set_title(f'{feature} by Cluster', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Cluster')
        axes[idx].set_ylabel(feature)
        plt.sca(axes[idx])
        plt.xticks(rotation=0)

# Remove empty subplot
fig.delaxes(axes[5])

plt.suptitle('Key Features Distribution Across K-Means Clusters', 
            fontsize=16, fontweight='bold', y=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# Radar chart for cluster profiles
from math import pi

# Select features for radar chart
radar_features = ['BALANCE', 'PURCHASES', 'CREDIT_LIMIT', 'PAYMENTS', 'CASH_ADVANCE']
radar_features = [f for f in radar_features if f in df_clean.columns][:5]

# Normalize for radar chart
cluster_profiles_norm = df_clustered.groupby('KMeans_Cluster')[radar_features].mean()
cluster_profiles_norm = (cluster_profiles_norm - cluster_profiles_norm.min()) / \
                        (cluster_profiles_norm.max() - cluster_profiles_norm.min())

# Number of variables
categories = list(cluster_profiles_norm.columns)
N = len(categories)

# Angle of each axis
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Plot
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))

for cluster in range(optimal_k):
    values = cluster_profiles_norm.loc[cluster].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {cluster}')
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12)
ax.set_ylim(0, 1)
ax.set_title('Cluster Profiles - Radar Chart (Normalized)', 
            fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

## 9. Cluster Comparison

In [ ]:
# Compare clustering methods
comparison_df = pd.DataFrame({
    'Method': ['K-Means', 'Hierarchical', 'DBSCAN'],
    'N_Clusters': [
        optimal_k, 
        optimal_k,
        n_clusters
    ],
    'Silhouette_Score': [
        silhouette_avg,
        silhouette_hier,
        silhouette_dbscan if n_clusters > 1 else np.nan
    ],
    'Calinski_Harabasz': [
        calinski,
        calinski_hier,
        np.nan  # Not calculated for DBSCAN with noise
    ],
    'Davies_Bouldin': [
        davies_bouldin,
        davies_bouldin_hier,
        np.nan  # Not calculated for DBSCAN with noise
    ]
})

print("CLUSTERING METHODS COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("\nNote: Higher Silhouette and Calinski-Harabasz scores are better.")
print("      Lower Davies-Bouldin score is better.")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['Silhouette_Score', 'Calinski_Harabasz', 'Davies_Bouldin']
titles = ['Silhouette Score (Higher is Better)', 
         'Calinski-Harabasz Score (Higher is Better)',
         'Davies-Bouldin Score (Lower is Better)']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    data = comparison_df.dropna(subset=[metric])
    axes[idx].bar(data['Method'], data[metric], edgecolor='black', alpha=0.7)
    axes[idx].set_title(title, fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Score')
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(data[metric]):
        axes[idx].text(i, v, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 10. Business Insights and Customer Segmentation

In [ ]:
# Detailed cluster interpretation (using K-Means clusters)
print("CUSTOMER SEGMENT INSIGHTS")
print("="*80)

for cluster in range(optimal_k):
    print(f"\n{'='*80}")
    print(f"CLUSTER {cluster} - Customer Profile")
    print(f"{'='*80}")
    
    cluster_data = df_clustered[df_clustered['KMeans_Cluster'] == cluster]
    print(f"\nSize: {len(cluster_data)} customers ({len(cluster_data)/len(df_clustered)*100:.1f}%)")
    
    print("\nKey Characteristics:")
    for col in ['BALANCE', 'PURCHASES', 'CREDIT_LIMIT', 'PAYMENTS']:
        if col in cluster_data.columns:
            mean_val = cluster_data[col].mean()
            overall_mean = df_clustered[col].mean()
            diff = ((mean_val - overall_mean) / overall_mean) * 100
            print(f"  {col}: ${mean_val:,.2f} ({diff:+.1f}% vs overall avg)")
    
    # Purchase behavior
    if 'PURCHASES_FREQUENCY' in cluster_data.columns:
        print(f"\n  Purchase Frequency: {cluster_data['PURCHASES_FREQUENCY'].mean():.3f}")
    if 'ONEOFF_PURCHASES' in cluster_data.columns and 'INSTALLMENTS_PURCHASES' in cluster_data.columns:
        one_off = cluster_data['ONEOFF_PURCHASES'].mean()
        installment = cluster_data['INSTALLMENTS_PURCHASES'].mean()
        print(f"  One-off Purchases: ${one_off:,.2f}")
        print(f"  Installment Purchases: ${installment:,.2f}")

In [ ]:
# Create segment names based on characteristics
def assign_segment_name(row):
    cluster = row['KMeans_Cluster']
    balance = row['BALANCE'] if 'BALANCE' in row else 0
    purchases = row['PURCHASES'] if 'PURCHASES' in row else 0
    
    # Simple heuristic - adjust based on actual cluster characteristics
    if balance > df_clean['BALANCE'].quantile(0.75) and purchases > df_clean['PURCHASES'].quantile(0.75):
        return 'High-Value Active'
    elif balance < df_clean['BALANCE'].quantile(0.25) and purchases < df_clean['PURCHASES'].quantile(0.25):
        return 'Low-Activity'
    elif purchases > df_clean['PURCHASES'].quantile(0.5):
        return 'Regular Spenders'
    else:
        return 'Moderate Users'

df_clustered['Segment_Name'] = df_clustered.apply(assign_segment_name, axis=1)

print("\nCUSTOMER SEGMENT DISTRIBUTION")
print("="*80)
segment_dist = df_clustered['Segment_Name'].value_counts()
print(segment_dist)
print(f"\nPercentage distribution:")
print((segment_dist / len(df_clustered) * 100).round(2))

In [ ]:
# Segment distribution visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Pie chart
segment_counts = df_clustered['Segment_Name'].value_counts()
colors = plt.cm.Set3(range(len(segment_counts)))
ax1.pie(segment_counts, labels=segment_counts.index, autopct='%1.1f%%',
       startangle=90, colors=colors, textprops={'fontsize': 11, 'fontweight': 'bold'})
ax1.set_title('Customer Segment Distribution', fontsize=14, fontweight='bold')

# Bar chart with cluster mapping
cluster_segment = df_clustered.groupby(['KMeans_Cluster', 'Segment_Name']).size().unstack(fill_value=0)
cluster_segment.plot(kind='bar', ax=ax2, stacked=True, color=colors, edgecolor='black')
ax2.set_title('Segment Distribution by K-Means Cluster', fontsize=14, fontweight='bold')
ax2.set_xlabel('K-Means Cluster')
ax2.set_ylabel('Number of Customers')
ax2.legend(title='Segment', bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Save Results

In [ ]:
# Prepare results for export
results_df = df_clean.copy()
results_df['Customer_ID'] = customer_ids.values
results_df['KMeans_Cluster'] = kmeans_labels
results_df['Hierarchical_Cluster'] = hierarchical_labels
results_df['DBSCAN_Cluster'] = dbscan_labels
results_df['Segment_Name'] = df_clustered['Segment_Name'].values

# Reorder columns
cols = ['Customer_ID', 'KMeans_Cluster', 'Hierarchical_Cluster', 'DBSCAN_Cluster', 'Segment_Name'] + \
       [col for col in results_df.columns if col not in ['Customer_ID', 'KMeans_Cluster', 
                                                          'Hierarchical_Cluster', 'DBSCAN_Cluster', 
                                                          'Segment_Name']]
results_df = results_df[cols]

# Save to CSV
output_path = '../../../data/outputs/customer_segments.csv'
results_df.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")

# Save cluster profiles
cluster_profiles_path = '../../../data/outputs/cluster_profiles.csv'
cluster_profiles.to_csv(cluster_profiles_path)
print(f"Cluster profiles saved to: {cluster_profiles_path}")

# Save comparison metrics
comparison_path = '../../../data/outputs/clustering_comparison.csv'
comparison_df.to_csv(comparison_path, index=False)
print(f"Comparison metrics saved to: {comparison_path}")

print("\nAll results saved successfully!")

## 12. Summary and Key Findings

In [ ]:
print("="*80)
print("SESSION 7 SUMMARY: CLUSTERING ANALYSIS")
print("="*80)

print("\n1. DATASET")
print(f"   - Total customers analyzed: {len(df_clean):,}")
print(f"   - Features used: {len(df_clean.columns)}")
print(f"   - Data quality: {(1 - df_clean.isnull().sum().sum() / df_clean.size) * 100:.2f}% complete")

print("\n2. CLUSTERING METHODS APPLIED")
print(f"   - K-Means: {optimal_k} clusters")
print(f"   - Hierarchical: {optimal_k} clusters (Ward linkage)")
print(f"   - DBSCAN: {n_clusters} clusters + {n_noise} noise points")

print("\n3. BEST PERFORMING METHOD")
best_method = comparison_df.loc[comparison_df['Silhouette_Score'].idxmax()]
print(f"   - Method: {best_method['Method']}")
print(f"   - Silhouette Score: {best_method['Silhouette_Score']:.4f}")
print(f"   - Calinski-Harabasz Score: {best_method['Calinski_Harabasz']:.2f}")

print("\n4. CUSTOMER SEGMENTS IDENTIFIED")
for idx, (segment, count) in enumerate(segment_dist.items(), 1):
    pct = (count / len(df_clustered)) * 100
    print(f"   {idx}. {segment}: {count:,} customers ({pct:.1f}%)")

print("\n5. KEY INSIGHTS")
print("   - PCA reduced dimensionality while retaining most variance")
print("   - Clear customer segments emerged based on spending behavior")
print("   - Different clustering methods showed consistent patterns")
print("   - DBSCAN identified outlier customers (potential fraud or VIP)")

print("\n6. BUSINESS RECOMMENDATIONS")
print("   - Target high-value segments with premium offerings")
print("   - Re-engage low-activity customers with incentives")
print("   - Customize marketing strategies per segment")
print("   - Monitor segment transitions for early intervention")

print("\n7. FILES GENERATED")
print("   - customer_segments.csv: Full results with cluster assignments")
print("   - cluster_profiles.csv: Mean characteristics per cluster")
print("   - clustering_comparison.csv: Performance metrics comparison")

print("\n" + "="*80)
print("Analysis complete! Ready for business implementation.")
print("="*80)